In [1]:
%pip install fiftyone

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.6/133.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.7/74.7 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.5/192.5 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## 1) Data Loading from train set (25 pcs, class = "Flowers")

In [3]:
import fiftyone as fo
import fiftyone.zoo as foz
import pandas as pd

classes = ["Flower"]
num_images = 25
dataset_directory = './dataset'

dataset = foz.load_zoo_dataset(
    "open-images-v7",
    split="train",
    label_types=["detections"],
    dataset_dir=dataset_directory,
    classes=classes,
    max_samples=num_images,
)
images_dir = f"{dataset_directory}/train/data"
labels_path = f"{dataset_directory}/train/labels/detections.csv"


INFO:fiftyone.zoo.datasets:Downloading split 'train' to './dataset/train' if necessary


KeyboardInterrupt: 

In [15]:
import shutil

target_dir = f"{dataset_directory}/test/originals"

# Create the target directory if it doesn't exist
os.makedirs(target_dir, exist_ok=True)

# Copy images to the target directory
for sample in dataset:
    source_path = sample.filepath
    filename = os.path.basename(source_path)
    target_path = os.path.join(target_dir, filename)
    shutil.copy(source_path, target_path)

# Copy the labels file to the target directory
target_labels_path = os.path.join(target_dir, 'detections.csv')
shutil.copy(labels_path, target_labels_path)

'./dataset/test/originals/detections.csv'

## 2) Data Loading from test set (25 pcs, class = "Flowers")

In [6]:
import fiftyone as fo
import fiftyone.zoo as foz
import pandas as pd

classes = ["Flower"]
num_images = 25
dataset_directory = './dataset'

dataset = foz.load_zoo_dataset(
    "open-images-v7",
    split="validation",
    label_types=["detections"],
    dataset_dir=dataset_directory,
    classes=classes,
    max_samples=num_images,
)
images_dir = f"{dataset_directory}/validation/data"
labels_path = f"{dataset_directory}/validation/labels/detections.csv"
target_dir = f"{dataset_directory}/test/originals"

INFO:fiftyone.zoo.datasets:Downloading split 'validation' to './dataset/validation' if necessary


Necessary images already downloaded


INFO:fiftyone.utils.openimages:Necessary images already downloaded


Existing download of split 'validation' is sufficient


INFO:fiftyone.zoo.datasets:Existing download of split 'validation' is sufficient


Loading existing dataset 'open-images-v7-validation-25'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


INFO:fiftyone.zoo.datasets:Loading existing dataset 'open-images-v7-validation-25'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


NameError: name 'os' is not defined

In [16]:
# Copy images to the target directory
for sample in dataset:
    source_path = sample.filepath
    filename = os.path.basename(source_path)
    target_path = os.path.join(target_dir, filename)
    shutil.copy(source_path, target_path)

# Copy the labels file to the target directory
target_labels_path = os.path.join(target_dir, 'detections.csv')
shutil.copy(labels_path, target_labels_path)

'./dataset/test/originals/detections.csv'

## 1) Window Sliding for train part

In [11]:
import cv2
import os
import random
from tqdm import tqdm
from PIL import Image
import pandas as pd

global_component_counter = 0
components_dict = {}

# Function implements sliding window technique
def sliding_window(image, step_size, window_size):
    for y in range(0, image.shape[0] - window_size[1] + 1, step_size):
        for x in range(0, image.shape[1] - window_size[0] + 1, step_size):
            yield (x, y, image[y:y + window_size[1], x:x + window_size[0]])

# Function to save image components as PIL images and store in array
def save_components(image_path, window_size, output_folder):
    global global_component_counter
    image = cv2.imread(image_path)
    components = []

    component_count = 0
    while component_count < 10:
        step_size = random.randint(0, 300)
        for (x, y, window) in sliding_window(image, step_size, window_size):
            pil_image = Image.fromarray(cv2.cvtColor(window, cv2.COLOR_BGR2RGB))
            components.append(pil_image)

            global_component_counter += 1
            component_count += 1
            if component_count >= 10:
                component = random.choice(components)

                # Save each component image in the output folder
                output_filename = f"{os.path.splitext(os.path.basename(image_path))[0]}_comp.jpg"
                output_path = os.path.join(output_folder, output_filename)
                pil_image.save(output_path)

                component_info = {
                    "Original_image": os.path.basename(image_path),
                    "Component": output_filename,
                    "Method": "WindowSlidingCut",
                    "Component_size": pil_image.size,
                    "Image_size": image.shape[:2]  # (height, width)
                }
                return component_info

            step_size = random.randint(0, 300)

# Function to process images folder
def process_images_folder(input_folder, window_size):
    global components_dict

    output_folder = '/content/dataset/test/fragments'
    os.makedirs(output_folder, exist_ok=True)

    data = []

    for filename in tqdm(os.listdir(input_folder)):
        if filename.lower().endswith('.jpg'):
            input_image_path = os.path.join(input_folder, filename)

            component_info = save_components(input_image_path, window_size, output_folder)
            if component_info:
                data.append(component_info)

            components_dict[filename] = component_info["Component"]

    return data

input_folder = '/content/dataset/train/data'
window_size = (300, 300)

component_data = process_images_folder(input_folder, window_size)

df_train = pd.DataFrame(component_data)

# Save to a csv file
csv_path = "DatasetWindowSlidingCut.csv"
df_train.to_csv(csv_path, index=False)

print(f"DataFrame saved to: {csv_path}")

100%|██████████| 25/25 [00:00<00:00, 56.27it/s]

DataFrame saved to: DatasetWindowSlidingCut.csv


## 2) Window Sliding for test part

In [13]:
import cv2
import os
import random
from tqdm import tqdm
from PIL import Image
import pandas as pd

global_component_counter = 0
components_dict = {}

# Function implements sliding window technique
def sliding_window(image, step_size, window_size):
    for y in range(0, image.shape[0] - window_size[1] + 1, step_size):
        for x in range(0, image.shape[1] - window_size[0] + 1, step_size):
            yield (x, y, image[y:y + window_size[1], x:x + window_size[0]])

# Function to save image components as PIL images and store in array
def save_components(image_path, window_size, output_folder):
    global global_component_counter
    image = cv2.imread(image_path)
    components = []

    component_count = 0
    while component_count < 10:
        step_size = random.randint(0, 300)
        for (x, y, window) in sliding_window(image, step_size, window_size):
            pil_image = Image.fromarray(cv2.cvtColor(window, cv2.COLOR_BGR2RGB))
            components.append(pil_image)

            global_component_counter += 1
            component_count += 1
            if component_count >= 10:
                component = random.choice(components)

                # Save each component image in the output folder
                output_filename = f"{os.path.splitext(os.path.basename(image_path))[0]}_comp.jpg"
                output_path = os.path.join(output_folder, output_filename)
                pil_image.save(output_path)

                component_info = {
                    "Original_image": os.path.basename(image_path),
                    "Component": output_filename,
                    "Method": "WindowSlidingCut",
                    "Component_size": pil_image.size,
                    "Image_size": image.shape[:2]  # (height, width)
                }
                return component_info

            step_size = random.randint(0, 300)

# Function to process images folder
def process_images_folder(input_folder, window_size):
    global components_dict

    output_folder = '/content/dataset/test/fragments'
    os.makedirs(output_folder, exist_ok=True)

    data = []

    for filename in tqdm(os.listdir(input_folder)):
        if filename.lower().endswith('.jpg'):
            input_image_path = os.path.join(input_folder, filename)

            component_info = save_components(input_image_path, window_size, output_folder)
            if component_info:
                data.append(component_info)

            components_dict[filename] = component_info["Component"]

    return data

input_folder = '/content/dataset/validation/data'
window_size = (300, 300)

component_data = process_images_folder(input_folder, window_size)

df_test = pd.DataFrame(component_data)
if os.path.exists(csv_path):
    existing_df = pd.read_csv(csv_path)
    combined_df = pd.concat([existing_df, df_test], ignore_index=True)

else:
    combined_df = df_test
combined_df.to_csv(csv_path, index=False)

print(f"Combined DataFrame saved to: {csv_path}")

100%|██████████| 25/25 [00:00<00:00, 46.53it/s]

Combined DataFrame saved to: DatasetWindowSlidingCut.csv
